# Using PValue ~ lfcSE For DEseq2 (using IHW)

I am using the file generated in the previous step. This script was originally used for DEXseq so some of the variable names will say DEXseq related information but this can be applied to DEseq2.


## Library

In [ ]:
# Load the IHW library
library("IHW")

## Load In Important Files

### Load In Average and Standard Deviation

The path to the Average and Standard Deviation `.tsv` created using the normalized counts and is located in a subdirectory of the parent folder. The parent folder is manually defined. The subdirectory containing the averages tsv is called `4___Calculating_Average_And_Standard_Deviation`.

The file is called `Average_And_Standard_Deviation_Using_Normalized_Counts_DEseq2.tsv`.

Just give the parent directory to the folder containing the tsv.

In [ ]:
# Parent directory path
parent_directory <- "/path/to/your/parent/directory"


### Define Path To Averages TsV

In [ ]:
# Subdirectory and file name
subdirectory <- "4___Calculating_Average_And_Standard_Deviation"
file_name <- "Average_And_Standard_Deviation_Using_Normalized_Counts_DEseq2.tsv"

# Create the full path to the file
file_path <- file.path(parent_directory, subdirectory, file_name)

file_path

In [ ]:
# Read the data from the file
Averages_And_Standard_Deviations <- read.table(file_path, header = TRUE, sep = "\t", stringsAsFactors = FALSE)

# Display the first few rows of the data frame
head(Averages_And_Standard_Deviations)

### Load in Wald Test TSV

The results are from the Wald test file from DEseq2 (not DEXseq).

In [ ]:
# Subdirectory containing the Wald test file
wald_test_subdirectory <- "1___Wald_Test"

# Pattern for the Wald test file
wald_test_file_pattern <- "^Wald_Test.*\\.tsv$"

# Create the full path to the Wald test file
wald_test_files <- list.files(file.path(parent_directory, wald_test_subdirectory), pattern = wald_test_file_pattern, full.names = TRUE)

# Check if any files were found
if (length(wald_test_files) > 0) {
  # Display the full path to the first matching Wald test file
  cat("Full path to the Wald test file:", wald_test_files[1], "\n")

  # Read the data from the file
  wald_test_data <- read.table(wald_test_files[1], header = TRUE, sep = "\t", stringsAsFactors = FALSE)

  # Display the first few rows of the data frame
  head(wald_test_data)
} else {
  cat("No Wald test files found in the specified directory.\n")
}

In [ ]:

wald_test_data$Ensembl_ID <- row.names(wald_test_data)

head (wald_test_data)


## Merge the 2 Important Files Together

In [ ]:
joined_df <- merge(Averages_And_Standard_Deviations, wald_test_data, by.x = "Ensembl_ID", 
             by.y = "Ensembl_ID", all.x = TRUE, all.y = FALSE)

head(joined_df)

In [ ]:
# Subset the data frame to include only specific columns
subset_df___Row_Standard_Deviation <- joined_df[, c("pvalue", "baseMean", "Row_Standard_Deviation")]

# Print the first few rows of the subsetted data frame
head(subset_df___Row_Standard_Deviation)

## Pvalue_Vs_BaseMeanAndRowStandardDeviation & IHW

In [ ]:
ihwRes_Pvalue_Vs_BaseMeanAndRowStandardDeviation <- ihw(pvalue ~ baseMean + Row_Standard_Deviation,  data = joined_df, alpha = 0.05)

Let us see the number of rejections:

In [ ]:
rejections(ihwRes_Pvalue_Vs_BaseMeanAndRowStandardDeviation)

In [ ]:
joined_df$ihwRes_Pvalue_Vs_BaseMeanAndRowStandardDeviation <- adj_pvalues(ihwRes_Pvalue_Vs_BaseMeanAndRowStandardDeviation)

head(joined_df)

# Create A Step 5 Folder:

In [ ]:
# New folder name
new_folder_name <- "5___Pvalue_Vs_BaseMeanAndRowStandardDeviation_IHW__And_DEseq2"

# Full path to the new folder
new_folder_path <- file.path(parent_directory, new_folder_name)

# Create the new folder
dir.create(new_folder_path, showWarnings = FALSE)

# Check if the folder was created successfully
if (file.exists(new_folder_path)) {
  cat("Folder", new_folder_name, "created successfully in", parent_directory, "\n")
} else {
  cat("Error creating the folder", new_folder_name, "in", parent_directory, "\n")
}

### Writing Joined DF to TSV and CSV File(s)

In [ ]:
# Assuming joined_df is your data frame

# File paths for TSV and CSV files
tsv_file <- file.path(new_folder_path, "Pvalue_Vs_BaseMeanAndRowStandardDeviation.tsv")
csv_file <- file.path(new_folder_path, "Pvalue_Vs_BaseMeanAndRowStandardDeviation.csv")

# Write to TSV file
write.table(joined_df, file = tsv_file, sep = "\t", quote = FALSE, row.names = FALSE)

# Write to CSV file
write.csv(joined_df, file = csv_file, quote = FALSE, row.names = FALSE)

# Print messages
cat("Data written to", tsv_file, "and", csv_file, "\n")


## Data Visualization:

### 2.2 Diagnostic Plots

#### 2.2.1 Estimated Weight(s)

In [ ]:
plot(ihwRes_Pvalue_Vs_BaseMeanAndRowStandardDeviation)

### 2.2.2 Decision boundary

In [ ]:
plot(ihwRes_Pvalue_Vs_BaseMeanAndRowStandardDeviation, what = "decisionboundary") 

### 2.2.3 Raw versus adjusted p-values

In [ ]:
library("ggplot2")
gg <- ggplot(as.data.frame(ihwRes_Pvalue_Vs_BaseMeanAndRowStandardDeviation), aes(x = pvalue, y = adj_pvalue, col = group)) + 
  geom_point(size = 0.25) + scale_colour_hue(l = 70, c = 150, drop = FALSE)
gg

In [ ]:
gg %+% subset(as.data.frame(ihwRes_Pvalue_Vs_BaseMeanAndRowStandardDeviation), adj_pvalue <= 0.05)

## 3.4 Diagnostic plots for the covariate

This is based on the code provided by IHW [here](https://bioconductor.org/packages/devel/bioc/vignettes/IHW/inst/doc/introduction_to_ihw.html#diagnostic-plots-for-the-covariate)

### Making Summarized Experiment:

In [ ]:
library(SummarizedExperiment)

The counts matrix used below has the normalized counts as those are prefered for the plots:


In [ ]:
# Subfolder name
subfolder_name <- "2___Normalized_Counts_DEseq2"

# File name
file_name <- "Normalized_Counts.tsv"

# Construct the path
normalized_matrix <- file.path(parent_directory, subfolder_name, file_name)

# Print the path
cat("Path to Normalized_Counts.tsv:", normalized_matrix, "\n")


# Read in the file
normalized_matrix <- read.table(normalized_matrix, header = TRUE, sep = '\t')


View the head of the normalized matrix:

In [ ]:
head(normalized_matrix)

In [ ]:
# Check for 0s in any row
rows_with_zeros <- apply(normalized_matrix[, -1], 1, function(row) any(row == 0))

# Remove rows with 0s
normalized_matrix <- normalized_matrix[!rows_with_zeros, ]


In [ ]:
head(normalized_matrix)

In [ ]:
colnames(normalized_matrix)

Turn the Ensembl ID column into row names:

In [ ]:
# Assuming your data frame is named df
rownames(normalized_matrix) <- normalized_matrix$Ensembl_ID
normalized_matrix$Ensembl_ID <- NULL  # Optional: Remove the Ensembl_ID column if you don't need it as a separate column
head(normalized_matrix)

In [ ]:
sample_metadata <- data.frame(
  Sample = c("C.01__Control",
             "C.02__Control",
             "C.03__Control",
             "E.01__Experimental",
             "E.02__Experimental",
             "E.03__Experimental"),
    
  Treatment = c("Untreated", "Untreated", "Untreated",
                "Knockdown", "Knockdown", "Knockdown")
)

# View the table

sample_metadata


## Creating A Summarized Experiment

First you need to check the dimensions of the count matrix:

In [ ]:
dim(normalized_matrix)


The columns can only be the names of the samples in this case so you may need to isolate for the sample columns/rename when necessary:

In [ ]:
se <- SummarizedExperiment(assays = list(counts = normalized_matrix), colData = sample_metadata)

In [ ]:
se

### 3.4.1 Scatter plots

In [ ]:
library("dplyr")

Define the path to averages file:

In [ ]:
# Subfolder name
avg_subfolder_name <- "4___Calculating_Average_And_Standard_Deviation"

# File name
file_name <- "Average_And_Standard_Deviation_Using_Normalized_Counts_DEseq2.tsv"

# Construct the path
tsv_file <- file.path(parent_directory, avg_subfolder_name, file_name)

tsv_file

In [ ]:

# Read in the TSV file
average_standard_deviation <- read.table(tsv_file, header = TRUE, row.names = 1, sep = "\t")

# View the first few rows of the data frame
head(average_standard_deviation)


In [ ]:
library(dplyr)

# Assuming 'df' is your data frame
average_standard_deviation$Ensembl_ID <- rownames(average_standard_deviation)

rownames(average_standard_deviation) <- NULL  # Remove the row names

# Print the modified data frame
head(average_standard_deviation)

In [ ]:
# Merge by the 'Ensembl_ID' column
merged_df <- merge(average_standard_deviation, wald_test_data, by.x = "Ensembl_ID", by.y = "Ensembl_ID", all = TRUE)

# 'all = TRUE' includes all rows from both data frames in the result
# You can change it to 'all = FALSE' if you only want rows with matching Ensembl IDs

# Print the merged data frame
head(merged_df)

View the column names:

In [ ]:
colnames(merged_df)

In [ ]:
deRes <- na.omit(merged_df)
deRes$gene_id <- as.numeric(gsub("ENSG[+]*", "", rownames(deRes)))
head(deRes)

### Base Mean

In [ ]:
# set up data frame for ggplotting
rbind(data.frame(pvalue = deRes$pvalue, covariate = rank(deRes$baseMean)/nrow(deRes), 
                 covariate_type="base mean"),
      data.frame(pvalue = deRes$pvalue, covariate = rank(deRes$Row_Standard_Deviation)/nrow(deRes), 
                 covariate_type="Row_Standard_Deviation")) %>%
ggplot(aes(x = covariate, y = -log10(pvalue))) + geom_hex(bins = 100) + 
   facet_grid( . ~ covariate_type) + ylab(expression(-log[10]~p))

### 3.4.2 Stratified p-value histograms

In [ ]:
ggplot(deRes, aes(x = pvalue)) + geom_histogram(binwidth = 0.025, boundary = 0)

In [ ]:
deRes$baseMeanGroup <- groups_by_filter(deRes$baseMean, 8)

ggplot(deRes, aes(x=pvalue)) + 
  geom_histogram(binwidth = 0.025, boundary = 0) +
  facet_wrap( ~ baseMeanGroup, nrow = 2)

In [ ]:
deRes$Row_Standard_DeviationGroup <- groups_by_filter(deRes$Row_Standard_Deviation, 8)

ggplot(deRes, aes(x=pvalue)) + 
  geom_histogram(binwidth = 0.025, boundary = 0) +
  facet_wrap( ~ Row_Standard_DeviationGroup, nrow = 2)

In [ ]:
# Controls_Row_Standard_Deviation

deRes$Controls_Row_Standard_DeviationGroup <- groups_by_filter(deRes$Controls_Row_Standard_Deviation, 8)

ggplot(deRes, aes(x=pvalue)) + 
  geom_histogram(binwidth = 0.025, boundary = 0) +
  facet_wrap( ~ Controls_Row_Standard_DeviationGroup, nrow = 2)

In [ ]:
ggplot(deRes, aes(x = pvalue, col = baseMeanGroup)) + stat_ecdf(geom = "step") 

In [ ]:
ggplot(deRes, aes(x = pvalue, col = Row_Standard_DeviationGroup)) + stat_ecdf(geom = "step") 

In [ ]:
ggplot(deRes, aes(x = pvalue, col = Controls_Row_Standard_DeviationGroup)) + stat_ecdf(geom = "step") 

## Doing Plots Reccomended By Isabelle:

In [ ]:
head(joined_df)

### Base Mean

In [ ]:
#stratified histogram 
joined_df$baseMeanGroup <- groups_by_filter(joined_df$baseMean, 4)
ggplot(joined_df, aes(x = joined_df$pvalue)) + 
  geom_histogram() + facet_wrap(~baseMeanGroup)

In [ ]:
#scatter
ggplot(joined_df, aes(x=baseMean, y=-log10(joined_df$pvalue))) + geom_point()

In [ ]:
#empirical cumulative distribution functions (ECDF) 
joined_df$baseMeanGroup <- groups_by_filter(joined_df$baseMean, 4)
ggplot(joined_df, aes(x = pvalue, col = baseMeanGroup)) + stat_ecdf(geom = "step") 

## Standard Deviation

In [ ]:
joined_df$Row_Standard_DeviationGroup <- groups_by_filter(joined_df$Row_Standard_Deviation, 4)
ggplot(joined_df, aes(x = joined_df$pvalue)) + 
  geom_histogram() + facet_wrap(~Row_Standard_DeviationGroup)

In [ ]:
#scatter
ggplot(joined_df,
       aes(x=Row_Standard_Deviation, y=-log10(joined_df$pvalue))) + geom_point()

In [ ]:
#empirical cumulative distribution functions (ECDF) 
joined_df$Row_Standard_DeviationGroup <- groups_by_filter(joined_df$Row_Standard_Deviation, 4)
ggplot(joined_df, aes(x = pvalue, col = Row_Standard_DeviationGroup)) + stat_ecdf(geom = "step") 

### Log Fold Change Standard Error

In [ ]:
ggplot(joined_df, aes(x = joined_df$lfcSE)) + 
  geom_histogram() 

In [ ]:
joined_df$lfcSEGroup <- groups_by_filter(joined_df$lfcSE, 4)
ggplot(joined_df, aes(x = joined_df$pvalue)) + 
  geom_histogram() + facet_wrap(~lfcSEGroup)

In [ ]:
#scatter
ggplot(joined_df, aes(x=lfcSE, y=-log10(joined_df$pvalue))) + geom_point()

In [ ]:
#empirical cumulative distribution functions (ECDF) 
joined_df$lfcSEGroup <- groups_by_filter(joined_df$lfcSE, 4)
ggplot(joined_df, aes(x = pvalue, col = lfcSEGroup)) + stat_ecdf(geom = "step") 

## Check for colinearity

A general guideline is that a VIF larger than 5 or 10 is large, indicating that the model has problems estimating the coefficient. 

In [ ]:
library(regclass)
#check multicollinearity between covariates
M <- lm(pvalue ~ baseMean + Row_Standard_Deviation + lfcSE,data=joined_df)
VIF(M)

## Sesssion Information

In [ ]:
sessionInfo()

In [ ]:
print("Done")